In [10]:
 '''
# Build Single Elliptical Axon

Kathleen Kish
22 October 2020

Function inputs: Axon starting coordinates (x,y,z)
                 Nerve fiber layer entry coordinates (x,y,z)
                 Total neuron length (L)

Function outputs: Axon coordinates (x,y,z) as a 3xL array

This function calculates the trajectory of an axon using basic geometric principles.
The axon arcs along the trajectory of a circle which contains both the soma starting coordinates
and the nerve fiber entry coordinates, and for which there is a horizontal tangent line at the nerve
fiber entry coordinates.
From there, it extends to the left for the remaining distance of the axon.

*Note: This code does not assign NEURON properties, it only calculates the x,y,z coordinates for the trajectories
*Note: ALL coordinates are in mm

'''
def unit(vector):
    a=vector[0]
    b=vector[1]
    c=vector[2]
    magnitude=((a**2)+(b**2)+(c**2))**0.5
    unit_vector=[a/magnitude,b/magnitude,c/magnitude]
    return unit_vector
    
def Build_Single_RGC(StartingCoordinates,NFLEntryCoordinates,Length):
    
    x_coords=np.array([]) # Array to store x-coordinates
    y_coords=np.array([]) # Array to store y-coordinates
    z_coords=np.array([]) # Array to store z-coordinates

    x_coords=np.append(x_coords,StartingCoordinates[0]) # The starting x-position of the soma is provided to the algortihm
    y_coords=np.append(y_coords,StartingCoordinates[1]) # The starting y-position of the soma is provided to the algorithm
    z_coords=np.append(z_coords,StartingCoordinates[2]) # The starting z-position of the soma is provided to the algorithm


    ###### BUILD ENTRANCE TO Nerve Fiber Layer
    
    # Find equation for arc connecting P and Q with tangent line at Q
    P = (x_coords[-1],y_coords[-1],z_coords[-1])
    Q = (NFLEntryCoordinates[0],NFLEntryCoordinates[1],NFLEntryCoordinates[2])
    
    # Find center of ellipse (h,k) in the x,z plane
    h = Q[0] 
    k = P[2]
    
    # Find major and minor axis of ellipse (a, b)
    a = P[0] - h
    b = Q[2] - k
    
    # Calculate points
    while x_coords[-1] > h:
        x_coords=np.append(x_coords,x_coords[-1]-0.001) # x position increments left by 1 um each step
        y_coords=np.append(y_coords,y_coords[-1]) # y position stays constant
        frac=((x_coords[-1]-h)**2)/(a**2)
        z_new=k+np.sqrt((b**2)*(1-frac))
        z_coords=np.append(z_coords,z_new) # z is calculated from the ellipse equation
    
    # Evenly distribute bend points (so they are always 4 micron apart)
    # Step 1 - Remove points that are too close together
    points_to_remove=[]
    for i in np.arange(len(x_coords)-1):
        x=x_coords[i+1]-x_coords[i]
        y=y_coords[i+1]-y_coords[i]
        z=z_coords[i+1]-z_coords[i]
        distance=np.sqrt((x**2)+(y**2)+(z**2)) # how far apart are two consecutive points?
        distance_from_one=distance-0.001 # how much different than 1 um? (+ = more than 1 um apart, - = less than 1 um apart)
        if distance_from_one<0:
            #print('points are too close')
            points_to_remove.append(i)
    x_coords=np.delete(x_coords,points_to_remove,0)
    y_coords=np.delete(y_coords,points_to_remove,0)
    z_coords=np.delete(z_coords,points_to_remove,0)
    
    # Step 2 - Redistribute points evenly
    X_Even=[] # New array to store the evenly spaced x values
    Y_Even=[] # New array to store the evenly spaced y values
    Z_Even=[] # New array to store the evenly spaced z values
    current_x=x_coords[0] # Starting x point is the beginning of the RGC
    current_y=y_coords[0] # Starting y point is the beginning of the RGC
    current_z=z_coords[0] # Starting z point is the beginning of the RGC
    i=0 # Loop to iterate through each point in the RGC
    while i < len(x_coords)-1:   
        X_Even.append(current_x) # Append current x point to the array
        Y_Even.append(current_y) # Append current y point to the array
        Z_Even.append(current_z) # Append current z point to the array
        next_x=x_coords[i+1] # Locate the next x point on the cell
        next_y=y_coords[i+1] # Locate the next y point on the cell
        next_z=z_coords[i+1] # Locate the next z point on the cell
        x=next_x-current_x # Find the distance between the current and next x point
        y=next_y-current_y # Find the distance between the current and next y point
        z=next_z-current_z # Find the distance between the current and next z point
        unit_vector=unit((x,y,z)) # Find the unit vector to define the direction between the two consecutive RGC points
        current_x=current_x+(.001*unit_vector[0]) # Define the new next point to be exactly 1 um away in the unit vector direction 
        current_y=current_y+(.001*unit_vector[1]) # Define the new next point to be exactly 1 um away in the unit vector direction 
        current_z=current_z+(.001*unit_vector[2])  # Define the new next point to be exactly 1 um away in the unit vector direction 
        i=i+1
    
    x_coords=X_Even
    y_coords=Y_Even
    z_coords=Z_Even

    
    ###### BUILD REMAINING AXON
    points_left = Length - len(x_coords) # how many points are needed to get cell to it's total length?
    for axonpts in range(points_left):
        x_coords=np.append(x_coords,x_coords[-1]-0.001) # x position moves increments left by 1 um each step
        y_coords=np.append(y_coords,y_coords[-1]) # y position stays constant
        z_coords=np.append(z_coords,z_coords[-1]) # z position stays constant
    
    RGC_coords=np.empty((0,3),int) # Final array to store all RGC coordinates   
    for i in np.arange(Length):
        Row=np.array([[x_coords[i],y_coords[i],z_coords[i]]])
        RGC_coords=np.append(RGC_coords,Row, axis=0)

    return(RGC_coords)